# 🛒 Alpha‑AGI Marketplace · Colab Notebook

Run the Alpha‑Factory marketplace micro-demo.

* 🔧 Start the orchestrator
* 📨 Queue a sample job and inspect memory
* 🔴 Works offline via Mixtral unless you provide an `OPENAI_API_KEY`

Paste your OpenAI key below or leave blank to run offline.

In [ ]:
OPENAI_API_KEY = ""  # optional

In [ ]:
%%bash
set -euo pipefail
REPO=AGI-Alpha-Agent-v0
if [ ! -d "$REPO/.git" ]; then
  echo '📥 Cloning repo ...'
  git clone --depth 1 https://github.com/MontrealAI/AGI-Alpha-Agent-v0.git $REPO
fi
cd $REPO

echo '🔧 Installing dependencies ...'
pip install -q -r alpha_factory_v1/requirements-colab.txt
pip install -q uvicorn fastapi pyngrok ctransformers==0.2.27

mkdir -p alpha_factory_v1/{keys,policies}
ssh-keygen -t ed25519 -N '' -q -f alpha_factory_v1/keys/agent_key <<<y || true
PUB=$(cat alpha_factory_v1/keys/agent_key.pub)

cat > alpha_factory_v1/.env <<EOF
OPENAI_API_KEY=${OPENAI_API_KEY}
OPENAI_API_BASE=${OPENAI_API_BASE:-https://api.openai.com/v1}
PROMPT_SIGN_PUBKEY=${PUB}
EOF

if [ -z "${OPENAI_API_KEY}" ]; then
  pip install -q sentencepiece
  python - <<'PY'
from ctransformers import AutoModelForCausalLM
AutoModelForCausalLM.from_pretrained(
  'TheBloke/Mixtral-8x7B-Instruct-GGML',
  model_file='mixtral-8x7b-instruct.ggmlv3.q4_K_M.bin',
  local_files_only=False)
PY
fi

## 🚀 Launch orchestrator (background)

In [ ]:
%%bash --bg
cd AGI-Alpha-Agent-v0/alpha_factory_v1
uvicorn backend.orchestrator:app --host 0.0.0.0 --port 8000 &>/dev/null &

## 🔗 Expose API with pyngrok

In [ ]:
from pyngrok import ngrok, conf
conf.get_default().region = 'us'
api = ngrok.connect(8000, 'http')
print('API →', api.public_url)

## ✅ Check orchestrator status

In [ ]:
from alpha_factory_v1.demos.alpha_agi_marketplace_v1 import MarketplaceClient
client = MarketplaceClient()
print('health →', client.health())
print('agents →', client.agents())

## 📋 Queue sample job and view memory

In [ ]:
from alpha_factory_v1.demos.alpha_agi_marketplace_v1 import MarketplaceClient, SAMPLE_JOB
import time, json
client = MarketplaceClient()
for _ in range(20):
    try:
        if client.health() == 'ok':
            break
    except Exception:
        time.sleep(1)

job = json.load(open(SAMPLE_JOB))
client.queue_job(job)
time.sleep(2)
print('recent memory →', client.recent_memory(job['agent']))

---
### 🎉 All set!
Use the API URL above to monitor progress or trigger your own jobs.